In [94]:
import numpy as np
from abc import ABC, abstractmethod
from dataclasses import dataclass

In [95]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
import models as mod
import points as pts

In [97]:
# get RNG
rng  = np.random.default_rng()

## Point processes & tools

In [98]:
# make a time window
window = pts.TimeWindow(0, 5, 2)

# specify arrival process
arrivals = pts.PoissonProcess(window, rate=0.3)

print(arrivals)

HomogeneousPoissonProcess(TimeWindow(0, 5, buffer=2), rate=0.3)


In [99]:
# sample arrival times
start = arrivals.sample(1, seed=rng)

print(start)

[array([-1.99557685,  2.51152366,  5.83633301,  4.00938081, -0.34030414])]


## WirelessTraffic

In [100]:
# sample Traffic parameters
nPackets = len(start[0])
airtime = rng.uniform(0.1, 0.2, size=nPackets)
channel = rng.choice(8, size=nPackets)
sf = 7 + rng.choice(4, size=nPackets)
power = rng.normal(loc=-90, scale=10, size=nPackets)

# make traffic object
traffic = mod.Traffic(nPackets, start[0], airtime,
                      channel, sf, power
                     )

print(traffic)

Traffic(nPackets=5)


## Network - A Traffic Generating Process

We can still keep the `LoRaWAN` class, just equip it with some distributions. These distributions should be customizable.
- Distribution over channels.
- Distribution over SFs.
- Distribution over payloads (s)

In [ ]:
still want a LoRaWAN just with 

# A Network class

In [123]:
WAN = mod.LoRaWAN()

print(WAN)

A LoRaWAN object with params:
	 nChannels: 1
	 freq: 915
	 bw: 125
	 sf: [7, 8, 9, 10]
	 overhead: 13
	 dwellTime: 400
	 dutyCycle: None


In [126]:
WAN

LoRaWAN({'nChannels': 1, 'freq': 915, 'bw': 125, 'sf': [7, 8, 9, 10], 'overhead': 13, 'dwellTime': 400, 'dutyCycle': None})

## Parameter Distribution

In [107]:
class distribution(ABC):
    """
    Class for custom made probability distributions.
    
    Notes
    -----
    
    The `__call__` function is used for sampling.
    The parameters and defaults should be explicit.
    Each distribution uses a `numpy.random.generator`,
    and should accept and optional `seed` to which
    a `seed` string or a `numpy.random.generator` can be
    passed.
    
    """
    
    @classmethod
    def __call__(self):
        pass

In [117]:
class Normal(distribution):
    """A normal distribution.
    """
    
    def __init__(self, loc=0, scale=1):
        """Instantiate normal distribution.
        """
        self.loc = loc
        self.scale = scale
        
    def __repr__(self):
        return "Normal(%s)" % str(self.__dict__) 
        
        
    def __call__(self, loc=None, scale=None, size=None, seed=None):
        """
        Generate a sample of normals.
        """ 
        if loc is None:
            loc = self.loc
        if scale is None:
            scale = self.scale
            
        rng = np.random.default_rng(seed)
        
        return rng.normal(loc=loc, scale=scale, size=size)

In [127]:
norm = Normal()

In [129]:
norm(loc=10, scale=5)

12.587114033153227

In [122]:
class WirelessDistribution(ABC):
    """
    A distribution over wireless parameters.
    
    Attributes
    ----------
    
    Notes
    -----
    
    This class models any distribution over wireless
    parameters, conditional on arrivals. I.e. the
    number of packets and their arrival times.
    """
    
    @classmethod
    def sample(self):
        pass    

In [106]:
class Independent(WirelessDistribution):
    """
    A distribution over LoRa Tx parameters in which
    each type of parameter are mutuall independent.
    
    Attributes
    ----------
    payloadDist
        A distribution over payloads.
    channelDist
        A distribution over channels.
    spreadingDidst
        A distribution over spreading factors.
    powerDist
        A distribution over power.
    """
    
    
    

In [30]:
class LoRaProcess():
    """
    A traffic-generating process.
    
    Attributes
    ----------
    network : LoRaWAN
        A LoRaWAN specifying wireless parameters of traffic.
    arrivals : ArrivalProcess
        Arrival process of wireless traffic.
    paramDist
        A distribution over network parameters.
        
    Notes
    -----
    This is a data-generating process. The main purpose of the class
    is to collect all relevant components in one place.
    
    A general `paramDist` is prefered over a distribution over individual
    wireless parameters, as the more general distribution permits
    arbitrary joint distrtibutions over groups of parameters. E.g. when
    power and SF are correlated.
    """
    
    def __init__(self):
        """
        Instantiate LoRaProcess.
        """
        